# **Visualisations**

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

## Functions

### DetectGPT Discrepancy Calculation

In [ ]:
def compute_detectgpt_discrepancy(log_probs_per_text_base: list,
                                log_probs_per_text_transformed: list,
                                normalization: bool=False) -> list:
    """
    Compute the DetectGPT discrepancy metric for each of the n_samples texts. Computed for n_perturbations perturbations.

    Args:
        log_probs_per_text_base (list): original log probability of each text
        log_probs_per_text_transformed (list): list of size n_samples where each element is a list of the n_perturbations perturbed log probs
        normalization (bool): True if you want to normalize the discrepancy scores, False otherwise

    Returns:
        discrepancy_scores (list): list of discrepancy values (d) for the n_samples texts
    """
    n_samples = len(log_probs_per_text_base) 
    discrepancy_scores = []

    for i in range(n_samples):
        original_log_prob = log_probs_per_text_base[i]
        perturbed_log_probs = log_probs_per_text_transformed[i] # List of perturbed log probs

        # Compute mean log probability of the perturbed texts
        mu = np.mean(perturbed_log_probs)  

        # Compute discrepancy
        discrepancy_score_unormalized = original_log_prob - mu
        if normalization:
            # Normalize
            sigma = np.std(perturbed_log_probs)
            discrepancy_score_normalized = discrepancy_score_unormalized / sigma if sigma > 0 else discrepancy_score_unormalized
            discrepancy_scores.append(discrepancy_score_normalized)
        else:
            discrepancy_scores.append(discrepancy_score_unormalized)
    
    return discrepancy_scores

### AUROC Calculation

In [ ]:
def get_roc_metrics(discrepancy_scores_human, discrepancy_scores_ai):
    fpr, tpr, _ = roc_curve(
    np.array([0] * len(discrepancy_scores_human) + [1] * len(discrepancy_scores_ai)),
    np.concatenate([discrepancy_scores_human, discrepancy_scores_ai])
    )
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)

### Histogram Grid

In [ ]:
def plot_histogram_grid(list_of_scores, title, row_titles, col_titles, row_heading, col_heading):
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    fig.suptitle(title, fontsize=24, fontweight='bold', y=1.04)  # Title further away
    
    for i, ax in enumerate(axes.flat):
        human_scores = list_of_scores[i * 2]
        ai_scores = list_of_scores[i * 2 + 1]

        fpr,tpr,roc_auc = get_roc_metrics(human_scores, ai_scores)

        ax.hist(human_scores, bins='auto', alpha=0.5, label='Human', edgecolor='black')
        ax.hist(ai_scores, bins='auto', alpha=0.5, label='AI', edgecolor='black')


        ax.set_title(f'AUROC {roc_auc:.2f}')

        if i % 3 == 0:
            ax.set_ylabel('Frequency')
            ax.text(-0.25, 0.5, row_titles[i // 3], transform=ax.transAxes, fontsize=16, fontweight = 'bold',
                    rotation=90, verticalalignment='center', horizontalalignment='center')

        if i < 3:
            ax.text(0.5, 1.15, col_titles[i], transform=ax.transAxes, fontsize=16, fontweight = 'bold',
                    rotation='horizontal', verticalalignment='bottom', horizontalalignment='center')

        if i >= 6:
            ax.set_xlabel('Discrepancy Score')
        
        ax.legend()
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    fig.text(-0.06, 0.5, row_heading, va='center', rotation='vertical', fontsize=20, fontweight = 'bold')  # Equidistant from grid
    fig.text(0.5, 0.95, col_heading, ha='center', fontsize=20, fontweight = 'bold')  # Centered relative to grid

    plt.show()

### AUROC Grid

In [ ]:
def plot_auroc_grid(list_of_scores, title, row_titles, col_titles, row_heading, col_heading):
    aurocs = [get_roc_metrics(list_of_scores[i * 2], list_of_scores[i * 2 + 1])[2] for i in range(9)]
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    auroc_matrix = np.array(aurocs).reshape(3, 3)
    sns.heatmap(auroc_matrix, annot=True, fmt='.2f', cmap='Blues', linewidths=0.5, ax=ax, cbar_kws={'label': 'AUROC'})

    ax.set_yticklabels(row_titles, fontsize=12, fontweight='bold', va='center', rotation=90)
    
    ax.set_xticklabels(col_titles, fontsize=12, fontweight='bold', ha='center', rotation=0)
    ax.xaxis.set_label_position('top')  # Move x-axis label (column titles) to the top
    ax.xaxis.tick_top()  # Ensure ticks and labels are on top
    
    # Set main title
    fig.suptitle(title, fontsize=16, fontweight='bold', y=1.08)

    # Set 'Scoring Model' and 'Generating Model' labels
    fig.text(0, 0.5, row_heading, va='center', rotation='vertical', fontsize=14, fontweight='bold')
    fig.text(0.5, 0.96, col_heading, ha='center', fontsize=14, fontweight='bold')

    plt.show()

## Data

### Import Data

In [ ]:
# Define the expected keys and structure
EXPECTED_KEYS = {"log_probs_base", "log_probs_transformed", "discrepancy_scores"}

def load_and_validate(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    # Check that the JSON object has exactly the three expected keys
    if set(data.keys()) != EXPECTED_KEYS:
        raise ValueError(f"Expected keys {EXPECTED_KEYS}, but got {set(data.keys())}")
    # Validate lengths for each list
    if len(data["log_probs_base"]) != 200:
        raise ValueError("log_probs_base should contain 200 floats")
    if len(data["log_probs_transformed"]) != 200:
        raise ValueError("log_probs_transformed should contain 200 lists")
    if len(data["discrepancy_scores"]) != 200:
        raise ValueError("discrepancy_scores should contain 200 floats")
    # Validate each inner list in log_probs_transformed has 100 floats
    for idx, inner_list in enumerate(data["log_probs_transformed"]):
        if len(inner_list) != 100:
            raise ValueError(f"Inner list at index {idx} in log_probs_transformed does not contain 100 floats")
    return data




# HUMAN TEXTS
# -------------------------------------------------------------------
# Generated by humans - GPT-J run
human_computed_by_gpt_j_gpt_j_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250318_171708.jsonl"
)

human_computed_by_gpt_neo_gpt_j_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250318_181503.jsonl"
)

human_computed_by_gpt_2_gpt_j_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250318_183528.jsonl"
)


# -------------------------------------------------------------------
# Generated by humans - GPT-Neo run
human_computed_by_gpt_j_gpt_neo_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250318_193811.jsonl"
)

human_computed_by_gpt_neo_gpt_neo_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250318_203938.jsonl"
)

human_computed_by_gpt_2_gpt_neo_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250318_210029.jsonl"
)


# -------------------------------------------------------------------
# Generated by humans - GPT-2 run
human_computed_by_gpt_j_gpt_2_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250319_103854.jsonl"
)

human_computed_by_gpt_neo_gpt_2_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250319_114542.jsonl"
)

human_computed_by_gpt_2_gpt_2_run_dct = load_and_validate(
    "/workspace/Results/XSUM_Human__200_Samples__200_Max_Length__100_Perturbations__Generated_By_Human__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250319_120520.jsonl"
)




# AI TEXTS
# -------------------------------------------------------------------
#  Generated by GPT-J
ai_generated_by_gpt_j_computed_by_gpt_j_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-j-6B__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250318_175629.jsonl"
)

ai_generated_by_gpt_j_computed_by_gpt_neo_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-j-6B__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250318_183320.jsonl"
)

ai_generated_by_gpt_j_computed_by_gpt_2_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-j-6B__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250318_183732.jsonl"
)


# -------------------------------------------------------------------
#  Generated by GPT-Neo
ai_generated_by_gpt_neo_computed_by_gpt_j_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-neo-2.7B__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250318_202111.jsonl"
)

ai_generated_by_gpt_neo_computed_by_gpt_neo_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-neo-2.7B__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250318_205825.jsonl"
)

ai_generated_by_gpt_neo_computed_by_gpt_2_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_EleutherAI/gpt-neo-2.7B__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250318_210233.jsonl"
)


# -------------------------------------------------------------------
#  Generated by GPT-2
ai_generated_by_gpt_2_computed_by_gpt_j_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_openai-community/gpt2__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-j-6B__20250319_112531.jsonl"
)

ai_generated_by_gpt_2_computed_by_gpt_neo_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_openai-community/gpt2__Perturbed_By_google-t5/t5-3b__Scored_By_EleutherAI/gpt-neo-2.7B__20250319_120321.jsonl"
)

ai_generated_by_gpt_2_computed_by_gpt_2_dct = load_and_validate(
    "/workspace/Results/XSUM_AI__200_Samples__200_Max_Length__100_Perturbations__Generated_By_openai-community/gpt2__Perturbed_By_google-t5/t5-3b__Scored_By_openai-community/gpt2__20250319_120717.jsonl"
)

## Calculate DetectGPT Discrepancies

### 1) GPT-J as Scorer

Set Model

In [ ]:
CACHE_DIR = "/tmp/huggingface"
COMPUTATION_MODEL_NAME = "EleutherAI/gpt-j-6B"
# Model list (all tested)

# openai-community/gpt2
# openai-community/gpt2-medium
# openai-community/gpt2-large
# openai-community/gpt2-xl

# EleutherAI/gpt-neo-2.7B
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b

computation_model_kwargs = {}
if 'gpt-j' in COMPUTATION_MODEL_NAME or 'neox' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(torch_dtype=torch.float16))
if 'gpt-j' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(revision='float16'))

# Load model
computation_model = AutoModelForCausalLM.from_pretrained(COMPUTATION_MODEL_NAME, **computation_model_kwargs, cache_dir=CACHE_DIR)

# Load tokenizer 
computation_tokenizer = AutoTokenizer.from_pretrained(COMPUTATION_MODEL_NAME, cache_dir=CACHE_DIR)

computation_tokenizer.model_max_length = 1024 

if computation_tokenizer.pad_token is None:
    computation_tokenizer.pad_token = computation_tokenizer.eos_token
computation_tokenizer.pad_token_id = computation_tokenizer.eos_token_id

computation_model.to(DEVICE)
print(DEVICE)

Calculate with Model

In [ ]:
# HUMAN TEXTS
human_computed_by_gpt_j_gpt_j_run = compute_detectgpt_discrepancy(human_computed_by_gpt_j_gpt_j_run_dct["log_probs_base"], human_computed_by_gpt_j_gpt_j_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_j_gpt_neo_run = compute_detectgpt_discrepancy(human_computed_by_gpt_j_gpt_neo_run_dct["log_probs_base"], human_computed_by_gpt_j_gpt_neo_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_j_gpt_2_run = compute_detectgpt_discrepancy(human_computed_by_gpt_j_gpt_2_run_dct["log_probs_base"], human_computed_by_gpt_j_gpt_2_run_dct["log_probs_transformed"], normalization=True)


# AI TEXTS
ai_generated_by_gpt_j_computed_by_gpt_j = compute_detectgpt_discrepancy(ai_generated_by_gpt_j_computed_by_gpt_j_dct["log_probs_base"], ai_generated_by_gpt_j_computed_by_gpt_j_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_neo_computed_by_gpt_j = compute_detectgpt_discrepancy(ai_generated_by_gpt_neo_computed_by_gpt_j_dct["log_probs_base"], ai_generated_by_gpt_neo_computed_by_gpt_j_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_2_computed_by_gpt_j = compute_detectgpt_discrepancy(ai_generated_by_gpt_2_computed_by_gpt_j_dct["log_probs_base"], ai_generated_by_gpt_2_computed_by_gpt_j_dct["log_probs_transformed"], normalization=True)

### 2) GPT-Neo as Scorer

Set Model

In [ ]:
CACHE_DIR = "/tmp/huggingface"
COMPUTATION_MODEL_NAME = "EleutherAI/gpt-neo-2.7B"
# Model list (all tested)

# openai-community/gpt2
# openai-community/gpt2-medium
# openai-community/gpt2-large
# openai-community/gpt2-xl

# EleutherAI/gpt-neo-2.7B
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b

computation_model_kwargs = {}
if 'gpt-j' in COMPUTATION_MODEL_NAME or 'neox' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(torch_dtype=torch.float16))
if 'gpt-j' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(revision='float16'))

# Load model
computation_model = AutoModelForCausalLM.from_pretrained(COMPUTATION_MODEL_NAME, **computation_model_kwargs, cache_dir=CACHE_DIR)

# Load tokenizer 
computation_tokenizer = AutoTokenizer.from_pretrained(COMPUTATION_MODEL_NAME, cache_dir=CACHE_DIR)

computation_tokenizer.model_max_length = 1024 

if computation_tokenizer.pad_token is None:
    computation_tokenizer.pad_token = computation_tokenizer.eos_token
computation_tokenizer.pad_token_id = computation_tokenizer.eos_token_id

computation_model.to(DEVICE)
print(DEVICE)

Calculate with Model

In [ ]:
# HUMAN TEXTS
human_computed_by_gpt_neo_gpt_j_run = compute_detectgpt_discrepancy(human_computed_by_gpt_neo_gpt_j_run_dct["log_probs_base"], human_computed_by_gpt_neo_gpt_j_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_neo_gpt_neo_run = compute_detectgpt_discrepancy(human_computed_by_gpt_neo_gpt_neo_run_dct["log_probs_base"], human_computed_by_gpt_neo_gpt_neo_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_neo_gpt_2_run = compute_detectgpt_discrepancy(human_computed_by_gpt_neo_gpt_2_run_dct["log_probs_base"], human_computed_by_gpt_neo_gpt_2_run_dct["log_probs_transformed"], normalization=True)


# AI TEXTS
ai_generated_by_gpt_j_computed_by_gpt_neo = compute_detectgpt_discrepancy(ai_generated_by_gpt_j_computed_by_gpt_neo_dct["log_probs_base"], ai_generated_by_gpt_j_computed_by_gpt_neo_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_neo_computed_by_gpt_neo = compute_detectgpt_discrepancy(ai_generated_by_gpt_neo_computed_by_gpt_neo_dct["log_probs_base"], ai_generated_by_gpt_neo_computed_by_gpt_neo_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_2_computed_by_gpt_neo = compute_detectgpt_discrepancy(ai_generated_by_gpt_2_computed_by_gpt_neo_dct["log_probs_base"], ai_generated_by_gpt_2_computed_by_gpt_neo_dct["log_probs_transformed"], normalization=True)

### 3) GPT-2 as Scorer

Set Model

In [ ]:
CACHE_DIR = "/tmp/huggingface"
COMPUTATION_MODEL_NAME = "openai-community/gpt2"
# Model list (all tested)

# openai-community/gpt2
# openai-community/gpt2-medium
# openai-community/gpt2-large
# openai-community/gpt2-xl

# EleutherAI/gpt-neo-2.7B
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b

computation_model_kwargs = {}
if 'gpt-j' in COMPUTATION_MODEL_NAME or 'neox' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(torch_dtype=torch.float16))
if 'gpt-j' in COMPUTATION_MODEL_NAME:
    computation_model_kwargs.update(dict(revision='float16'))

# Load model
computation_model = AutoModelForCausalLM.from_pretrained(COMPUTATION_MODEL_NAME, **computation_model_kwargs, cache_dir=CACHE_DIR)

# Load tokenizer 
computation_tokenizer = AutoTokenizer.from_pretrained(COMPUTATION_MODEL_NAME, cache_dir=CACHE_DIR)

computation_tokenizer.model_max_length = 1024 

if computation_tokenizer.pad_token is None:
    computation_tokenizer.pad_token = computation_tokenizer.eos_token
computation_tokenizer.pad_token_id = computation_tokenizer.eos_token_id

computation_model.to(DEVICE)
print(DEVICE)

Calculate with Model

In [ ]:
# HUMAN TEXTS
human_computed_by_gpt_2_gpt_j_run = compute_detectgpt_discrepancy(human_computed_by_gpt_2_gpt_j_run_dct["log_probs_base"], human_computed_by_gpt_2_gpt_j_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_2_gpt_neo_run = compute_detectgpt_discrepancy(human_computed_by_gpt_2_gpt_neo_run_dct["log_probs_base"], human_computed_by_gpt_2_gpt_neo_run_dct["log_probs_transformed"], normalization=True)

human_computed_by_gpt_2_gpt_2_run = compute_detectgpt_discrepancy(human_computed_by_gpt_2_gpt_2_run_dct["log_probs_base"], human_computed_by_gpt_2_gpt_2_run_dct["log_probs_transformed"], normalization=True)


# AI TEXTS
ai_generated_by_gpt_j_computed_by_gpt_2 = compute_detectgpt_discrepancy(ai_generated_by_gpt_j_computed_by_gpt_2_dct["log_probs_base"], ai_generated_by_gpt_j_computed_by_gpt_2_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_neo_computed_by_gpt_2 = compute_detectgpt_discrepancy(ai_generated_by_gpt_neo_computed_by_gpt_2_dct["log_probs_base"], ai_generated_by_gpt_neo_computed_by_gpt_2_dct["log_probs_transformed"], normalization=True)

ai_generated_by_gpt_2_computed_by_gpt_2 = compute_detectgpt_discrepancy(ai_generated_by_gpt_2_computed_by_gpt_2_dct["log_probs_base"], ai_generated_by_gpt_2_computed_by_gpt_2_dct["log_probs_transformed"], normalization=True)

### Organise Discrepancies

In [ ]:
scores_list = [
    human_computed_by_gpt_j_gpt_j_run, ai_generated_by_gpt_j_computed_by_gpt_j,   # Top-left
    human_computed_by_gpt_neo_gpt_j_run, ai_generated_by_gpt_j_computed_by_gpt_neo, # Top-middle
    human_computed_by_gpt_2_gpt_j_run, ai_generated_by_gpt_j_computed_by_gpt_2,   # Top-right
    human_computed_by_gpt_j_gpt_neo_run, ai_generated_by_gpt_neo_computed_by_gpt_j, # Middle-left
    human_computed_by_gpt_neo_gpt_neo_run, ai_generated_by_gpt_neo_computed_by_gpt_neo, # Middle
    human_computed_by_gpt_2_gpt_neo_run, ai_generated_by_gpt_neo_computed_by_gpt_2, # Middle-right
    human_computed_by_gpt_j_gpt_2_run, ai_generated_by_gpt_2_computed_by_gpt_j,   # Bottom-left
    human_computed_by_gpt_neo_gpt_2_run, ai_generated_by_gpt_2_computed_by_gpt_neo, # Bottom-middle
    human_computed_by_gpt_2_gpt_2_run, ai_generated_by_gpt_2_computed_by_gpt_2    # Bottom-right
    ]

## Execute Visualisation Code

In [ ]:
TITLE = 'Histograms of Discrepancy Scores'
ROW_TITLES = ["GPT-J", "GPT-Neo", "GPT-2"]
COL_TITLES = ["GPT-J", "GPT-Neo", "GPT-2"]
ROW_HEADING = "Generating Model"
COL_HEADING = "Scoring Model"

TITLE_AUROC = 'Grid of AUROCs'

plot_histogram_grid(scores_list, TITLE, ROW_TITLES, COL_TITLES, ROW_HEADING, COL_HEADING)
plot_auroc_grid(scores_list, TITLE_AUROC, ROW_TITLES, COL_TITLES, ROW_HEADING, COL_HEADING)

# **Ensemble Methods**

## Basic Statistics

For every entry in a scores list, you replace the entry with a summary statistic of that entry across multiple lists

### Mean

In [ ]:
def average_lists(list1, list2, list3):
    return [(x + y + z) / 3 for x, y, z in zip(list1, list2, list3)]

def average_lists_without_base(list1, list2):
    return [(x + y) / 2 for x, y in zip(list1, list2)]

Simple Mean

In [ ]:
# HUMAN TEXTS
# Calculate the arithmetic mean of the scores for the human text in the GPT-J run
human_scores_for_gpt_j_run_mean = average_lists(human_computed_by_gpt_j_gpt_j_run, human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the arithmetic mean of the scores for the human text in the GPT-Neo run
human_scores_for_gpt_neo_run_mean = average_lists(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_neo_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the arithmetic mean of the scores for the human text in the GPT-2 run
human_scores_for_gpt_2_run_mean = average_lists(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run, human_computed_by_gpt_2_gpt_2_run)


# AI TEXTS
# Check whether any of the lists of scores for the GPT-J generated texts are the same (note that each entry is a list)
assert not np.allclose(scores_list[1], scores_list[3], scores_list[5])
# Calculate the arithmetic mean of the scores for the GPT-J generated text (entries 1, 3, 5)
gpt_j_scores_mean = average_lists(ai_generated_by_gpt_j_computed_by_gpt_j, ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Check whether any of the lists of scores for the GPT-Neo generated texts are the same (note that each entry is a list)
assert not np.allclose(scores_list[7], scores_list[9], scores_list[11])
# Calculate the arithmetic mean of the scores for the GPT-Neo generated text (entries 7, 9, 11)
gpt_neo_scores_mean = average_lists(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_neo, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Check whether any of the lists of scores for the GPT-2 generated texts are the same (note that each entry is a list)
assert not np.allclose(scores_list[13], scores_list[15], scores_list[17])
# Calculate the arithmetic mean of the scores for the GPT-2 generated text (entries 13, 15, 17)
gpt_2_scores_mean = average_lists(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo, ai_generated_by_gpt_2_computed_by_gpt_2)

Mean Excluding Base Model

In [ ]:
# These calculate the mean, ignoring the results where the generating model is used as a scoring model

# HUMAN TEXTS
# Calculate the arithmetic mean of the scores for the human text in the GPT-J run, ignoring the GPT-J model
human_scores_for_gpt_j_run_mean_no_base = average_lists_without_base(human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the arithmetic mean of the scores for the human text in the GPT-Neo run, ignoring the GPT-Neo model
human_scores_for_gpt_neo_run_mean_no_base = average_lists_without_base(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the arithmetic mean of the scores for the human text in the GPT-2 run, ignoring the GPT-2 model
human_scores_for_gpt_2_run_mean_no_base = average_lists_without_base(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run)


# AI TEXTS
# Calculate the arithmetic mean of the scores for the GPT-J generated text, ignoring the GPT-J model
gpt_j_scores_mean_no_base = average_lists_without_base(ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Calculate the arithmetic mean of the scores for the GPT-Neo generated text, ignoring the GPT-Neo model
gpt_neo_scores_mean_no_base = average_lists_without_base(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Calculate the arithmetic mean of the scores for the GPT-2 generated text, ignoring the GPT-2 model
gpt_2_scores_mean_no_base = average_lists_without_base(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo)

### Median

In [ ]:
def median_lists(list1, list2, list3):
    return [np.median([x, y, z]) for x, y, z in zip(list1, list2, list3)]

def median_lists_without_base(list1, list2):
    return [np.median([x, y]) for x, y in zip(list1, list2)]

Simple Median

In [ ]:
# HUMAN TEXTS
# Calculate the median of the scores for the human text in the GPT-J run
human_scores_for_gpt_j_run_median= median_lists(human_computed_by_gpt_j_gpt_j_run, human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the median of the scores for the human text in the GPT-Neo run
human_scores_for_gpt_neo_run_median = median_lists(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_neo_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the median of the scores for the human text in the GPT-2 run
human_scores_for_gpt_2_run_median = median_lists(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run, human_computed_by_gpt_2_gpt_2_run)


# AI TEXTS
# Calculate the median of the scores for the GPT-J generated text (entries 1, 3, 5)
gpt_j_scores_median = median_lists(ai_generated_by_gpt_j_computed_by_gpt_j, ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Calculate the median of the scores for the GPT-Neo generated text (entries 7, 9, 11)
gpt_neo_scores_median = median_lists(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_neo, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Calculate the median of the scores for the GPT-2 generated text (entries 13, 15, 17)
gpt_2_scores_median = median_lists(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo, ai_generated_by_gpt_2_computed_by_gpt_2)

Median Excluding Base Model

In [ ]:
# These calculate the median, ignoring the results where the generating model is used as a scoring model

# HUMAN TEXTS
# Calculate the median of the scores for the human text in the GPT-J run, ignoring the GPT-J model
human_scores_for_gpt_j_run_median_no_base = median_lists_without_base(human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the median of the scores for the human text in the GPT-Neo run, ignoring the GPT-Neo model
human_scores_for_gpt_neo_run_median_no_base = median_lists_without_base(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the median of the scores for the human text in the GPT-2 run, ignoring the GPT-2 model
human_scores_for_gpt_2_run_median_no_base = median_lists_without_base(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run)


# AI TEXTS
# Calculate the median of the scores for the GPT-J generated text, ignoring the GPT-J model
gpt_j_scores_median_no_base = median_lists_without_base(ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Calculate the median of the scores for the GPT-Neo generated text, ignoring the GPT-Neo model
gpt_neo_scores_median_no_base = median_lists_without_base(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Calculate the median of the scores for the GPT-2 generated text, ignoring the GPT-2 model
gpt_2_scores_median_no_base = median_lists_without_base(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo)

### Maximum

In [ ]:
def max_lists(list1, list2, list3):
    return [max(x, y, z) for x, y, z in zip(list1, list2, list3)]

def max_lists_without_base(list1, list2):
    return [max(x, y) for x, y in zip(list1, list2)]

Simple Maximum

In [ ]:
# HUMAN TEXTS
# Calculate the maximum of the scores for the human text in the GPT-J run
human_scores_for_gpt_j_run_max = max_lists(human_computed_by_gpt_j_gpt_j_run, human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the maximum of the scores for the human text in the GPT-Neo run
human_scores_for_gpt_neo_run_max = max_lists(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_neo_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the maximum of the scores for the human text in the GPT-2 run
human_scores_for_gpt_2_run_max = max_lists(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run, human_computed_by_gpt_2_gpt_2_run)


# AI TEXTS
# Calculate the maximum of the scores for the GPT-J generated text (entries 1, 3, 5)
gpt_j_scores_max = max_lists(ai_generated_by_gpt_j_computed_by_gpt_j, ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Calculate the maximum of the scores for the GPT-Neo generated text (entries 7, 9, 11)
gpt_neo_scores_max = max_lists(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_neo, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Calculate the maximum of the scores for the GPT-2 generated text (entries 13, 15, 17)
gpt_2_scores_max = max_lists(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo, ai_generated_by_gpt_2_computed_by_gpt_2)

Maximum Excluding Base Model

In [ ]:
# These calculate the maximum, ignoring the results where the generating model is used as a scoring model

# HUMAN TEXTS
# Calculate the maximum of the scores for the human text in the GPT-J run, ignoring the GPT-J model
human_scores_for_gpt_j_run_max_no_base = max_lists_without_base(human_computed_by_gpt_neo_gpt_j_run, human_computed_by_gpt_2_gpt_j_run)

# Calculate the maximum of the scores for the human text in the GPT-Neo run, ignoring the GPT-Neo model
human_scores_for_gpt_neo_run_max_no_base = max_lists_without_base(human_computed_by_gpt_j_gpt_neo_run, human_computed_by_gpt_2_gpt_neo_run)

# Calculate the maximum of the scores for the human text in the GPT-2 run, ignoring the GPT-2 model
human_scores_for_gpt_2_run_max_no_base = max_lists_without_base(human_computed_by_gpt_j_gpt_2_run, human_computed_by_gpt_neo_gpt_2_run)


# AI TEXTS
# Calculate the maximum of the scores for the GPT-J generated text, ignoring the GPT-J model
gpt_j_scores_max_no_base = max_lists_without_base(ai_generated_by_gpt_j_computed_by_gpt_neo, ai_generated_by_gpt_j_computed_by_gpt_2)

# Calculate the maximum of the scores for the GPT-Neo generated text, ignoring the GPT-Neo model
gpt_neo_scores_max_no_base = max_lists_without_base(ai_generated_by_gpt_neo_computed_by_gpt_j, ai_generated_by_gpt_neo_computed_by_gpt_2)

# Calculate the maximum of the scores for the GPT-2 generated text, ignoring the GPT-2 model
gpt_2_scores_max_no_base = max_lists_without_base(ai_generated_by_gpt_2_computed_by_gpt_j, ai_generated_by_gpt_2_computed_by_gpt_neo)

### Organise Ensemble Data

Simple Ensembles

In [ ]:
scores_list_ensemble = [
    human_scores_for_gpt_j_run_mean, gpt_j_scores_mean,   # Top-left
    human_scores_for_gpt_j_run_median, gpt_j_scores_median,   # Top-middle
    human_scores_for_gpt_j_run_max, gpt_j_scores_max,   # Top-right
    human_scores_for_gpt_neo_run_mean, gpt_neo_scores_mean, # Middle-left
    human_scores_for_gpt_neo_run_median, gpt_neo_scores_median, # Middle
    human_scores_for_gpt_neo_run_max, gpt_neo_scores_max, # Middle-right
    human_scores_for_gpt_2_run_mean, gpt_2_scores_mean,   # Bottom-left
    human_scores_for_gpt_2_run_median, gpt_2_scores_median,   # Bottom-middle
    human_scores_for_gpt_2_run_max, gpt_2_scores_max    # Bottom-right
    ]

Ensembles Excluding Base Model

In [ ]:
scores_list_ensemble_no_base = [
    human_scores_for_gpt_j_run_mean_no_base, gpt_j_scores_mean_no_base,   # Top-left
    human_scores_for_gpt_j_run_median_no_base, gpt_j_scores_median_no_base,   # Top-middle
    human_scores_for_gpt_j_run_max_no_base, gpt_j_scores_max_no_base,   # Top-right
    human_scores_for_gpt_neo_run_mean_no_base, gpt_neo_scores_mean_no_base, # Middle-left
    human_scores_for_gpt_neo_run_median_no_base, gpt_neo_scores_median_no_base, # Middle
    human_scores_for_gpt_neo_run_max_no_base, gpt_neo_scores_max_no_base, # Middle-right
    human_scores_for_gpt_2_run_mean_no_base, gpt_2_scores_mean_no_base,   # Bottom-left
    human_scores_for_gpt_2_run_median_no_base, gpt_2_scores_median_no_base,   # Bottom-middle
    human_scores_for_gpt_2_run_max_no_base, gpt_2_scores_max_no_base    # Bottom-right
    ]

### Visualise

Simple Ensembles

In [ ]:
# The rows should be the generating models and the columns should be the ensemble methods (mean, median, max)

TITLE_ENSEMBLE = 'Histograms of Discrepancy Scores for Ensemble Methods'
ROW_TITLES_ENSEMBLE = ["GPT-J", "GPT-Neo", "GPT-2"]
COL_TITLES_ENSEMBLE = ["Mean", "Median", "Maximum"]
ROW_HEADING_ENSEMBLE = "Generating Model"
COL_HEADING_ENSEMBLE = "Ensemble Method"

TITLE_ENSEMBLE_AUROC = 'Grid of AUROCs for Ensemble Methods'

plot_histogram_grid(scores_list_ensemble, TITLE_ENSEMBLE, ROW_TITLES_ENSEMBLE, COL_TITLES_ENSEMBLE, ROW_HEADING_ENSEMBLE, COL_HEADING_ENSEMBLE)
plot_auroc_grid(scores_list_ensemble, TITLE_ENSEMBLE_AUROC, ROW_TITLES_ENSEMBLE, COL_TITLES_ENSEMBLE, ROW_HEADING_ENSEMBLE, COL_HEADING_ENSEMBLE)

Ensembles Excluding Base Model

In [ ]:
# The rows should be the generating models and the columns should be the ensemble methods (mean, median, max)

TITLE_ENSEMBLE_NO_BASE = 'Histograms of Discrepancy Scores for Ensemble Methods (Ignoring Base Model)'
ROW_TITLES_ENSEMBLE_NO_BASE = ["GPT-J", "GPT-Neo", "GPT-2"]
COL_TITLES_ENSEMBLE_NO_BASE = ["Mean", "Median", "Maximum"]
ROW_HEADING_ENSEMBLE_NO_BASE = "Generating Model"
COL_HEADING_ENSEMBLE_NO_BASE = "Ensemble Method"

TITLE_ENSEMBLE_AUROC_NO_BASE = 'Grid of AUROCs for Ensemble Methods (Ignoring Base Model)'
plot_histogram_grid(scores_list_ensemble_no_base, TITLE_ENSEMBLE_NO_BASE, ROW_TITLES_ENSEMBLE_NO_BASE, COL_TITLES_ENSEMBLE_NO_BASE, ROW_HEADING_ENSEMBLE_NO_BASE, COL_HEADING_ENSEMBLE_NO_BASE)
plot_auroc_grid(scores_list_ensemble_no_base, TITLE_ENSEMBLE_AUROC_NO_BASE, ROW_TITLES_ENSEMBLE_NO_BASE, COL_TITLES_ENSEMBLE_NO_BASE, ROW_HEADING_ENSEMBLE_NO_BASE, COL_HEADING_ENSEMBLE_NO_BASE)